<a href="https://colab.research.google.com/github/Jerrin-rajans/Coronavirus_India/blob/master/Indian_Coronavirus_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import folium

# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

from datetime import timedelta

#Dataset available at India Health Ministry Website https://www.mohfw.gov.in/ 
#Data collection methodology https://github.com/imdevskp/covid-19-india-data

In [44]:
# importing datasets

df = pd.read_csv('complete.csv', parse_dates=['Date'])
df.head()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged,Latitude,Longitude,Death
0,2020-01-30,Kerala,1,0,0,10.8505,76.2711,0
1,2020-01-31,Kerala,1,0,0,10.8505,76.2711,0
2,2020-02-01,Kerala,2,0,0,10.8505,76.2711,0
3,2020-02-02,Kerala,3,0,0,10.8505,76.2711,0
4,2020-02-03,Kerala,3,0,0,10.8505,76.2711,0


In [45]:
df.columns

Index(['Date', 'Name of State / UT', 'Total Confirmed cases (Indian National)',
       'Total Confirmed cases ( Foreign National )', 'Cured/Discharged',
       'Latitude', 'Longitude', 'Death'],
      dtype='object')

In [46]:
for i in ['Total Confirmed cases (Indian National)', 'Total Confirmed cases ( Foreign National )', 'Cured/Discharged', 'Death']:
    df[i] = df[i].astype('int')
    
df['Total cases'] = df['Total Confirmed cases (Indian National)'] + df['Total Confirmed cases ( Foreign National )']
df['Moratlity rate'] = df['Death']/df['Total cases']
df['Recovery rate'] = df['Cured/Discharged']/df['Total cases']

df.columns = ['Date', 'State/UT', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Cured', 'Latitude', 'Longitude', 'Death', 'Total cases', 'Mortality rate', 'Recovery rate']
df = df[['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Total cases', 'Death', 'Cured', 'Mortality rate', 'Recovery rate']]

df.head()

,Date,State/UT,Latitude,Longitude,Confirmed(Indian),Confirmed(Foreigner),Total cases,Death,Cured,Mortality rate,Recovery rate
0,2020-01-30,Kerala,10.8505,76.2711,1,0,1,0,0,0.0,0.0
1,2020-01-31,Kerala,10.8505,76.2711,1,0,1,0,0,0.0,0.0
2,2020-02-01,Kerala,10.8505,76.2711,2,0,2,0,0,0.0,0.0
3,2020-02-02,Kerala,10.8505,76.2711,3,0,3,0,0,0.0,0.0
4,2020-02-03,Kerala,10.8505,76.2711,3,0,3,0,0,0.0,0.0


In [0]:
latest = df[df['Date']==max(df['Date'])]

# days
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)

# state and total cases 
latest_cases = df[df['Date']==latest_day][['State/UT', 'Total cases']].set_index('State/UT')
db_cases = df[df['Date']==day_before][['State/UT', 'Total cases']].set_index('State/UT')

latest_cases.columns = ['lat_cases']
db_cases.columns = ['db_cases']

temp = pd.concat([latest_cases, db_cases], axis=1, sort=True).reset_index()
temp['New cases'] = temp['lat_cases']-temp['db_cases']

latest = pd.merge(left = latest, right=temp[['State/UT', 'New cases']], left_on='State/UT', right_on='State/UT', how='outer')

In [49]:
temp = latest[['State/UT', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Total cases', 'New cases',
               'Death', 'Mortality rate', 'Cured', 'Recovery rate',]]
temp = temp.sort_values('Confirmed(Indian)', ascending=False).reset_index(drop=True)

temp.style\
    .background_gradient(cmap="RdBu", subset=['Confirmed(Indian)', 'Confirmed(Foreigner)', 'Total cases', 'New cases'])\
    .background_gradient(cmap="Greens", subset=['Cured', 'Recovery rate'])\
    .background_gradient(cmap="Oranges", subset=['Death', 'Mortality rate'])

,State/UT,Confirmed(Indian),Confirmed(Foreigner),Total cases,New cases,Death,Mortality rate,Cured,Recovery rate
0,Maharashtra,36,3,39,7,1,0.025641,0,0
1,Kerala,24,2,26,3,0,0,3,0.115385
2,Uttar Pradesh,14,1,15,2,0,0,5,0.333333
3,Karnataka,11,0,11,5,1,0.0909091,0,0
4,Delhi,8,0,8,1,1,0.125,2,0.25
5,Union Territory of Ladakh,6,0,6,2,0,0,0,0
6,Telengana,3,2,5,2,0,0,1,0.2
7,Union Territory of Jammu and Kashmir,3,0,3,0,0,0,0,0
8,Rajasthan,2,2,4,0,0,0,3,0.75
9,Andhra Pradesh,1,0,1,0,0,0,0,0


In [38]:
m = folium.Map(location=[20.5937, 78.9629], tiles='cartodbpositron',
               min_zoom=4, max_zoom=6, zoom_start=4)

for i in range(0, len(latest)):
    if latest.iloc[i]['Confirmed(Indian)']>0:
        folium.Circle(
            location=[latest.iloc[i]['Latitude'], latest.iloc[i]['Longitude']],
            color='crimson', 
            tooltip =   '<li><bold>Name of State / UT : '+str(latest.iloc[i]['State/UT'])+
                        '<li><bold>Total Confirmed cases (Indian National) : '+str(latest.iloc[i]['Confirmed(Indian)'])+
                        '<li><bold>Total Confirmed cases ( Foreign National ) : '+str(latest.iloc[i]['Confirmed(Foreigner)']),
            radius=int(latest.iloc[i]['Confirmed(Indian)'])*3000).add_to(m)

for i in range(0, len(latest)):
    if latest.iloc[i]['Confirmed(Foreigner)']>0:
        folium.Circle(
            location=[latest.iloc[i]['Latitude'], latest.iloc[i]['Longitude']],
            color='purple', 
            tooltip =   '<li><bold>Name of State / UT : '+str(latest.iloc[i]['State/UT'])+
                        '<li><bold>Total Confirmed cases (Indian National) : '+str(latest.iloc[i]['Confirmed(Indian)'])+
                        '<li><bold>Total Confirmed cases ( Foreign National ) : '+str(latest.iloc[i]['Confirmed(Foreigner)']),
            radius=int(latest.iloc[i]['Confirmed(Foreigner)'])*3000).add_to(m)
m

In [39]:
temp = df.copy()
temp['Date'] = temp['Date'].dt.strftime('%Y/%m/%d')

fig = px.scatter_geo(temp, lat="Latitude", lon="Longitude", color='Total cases', size='Total cases',
                     hover_name="State/UT", scope='asia', animation_frame="Date", center={'lat':20, 'lon':78})
fig.show()

In [40]:
temp = df.melt(id_vars="Date", value_vars=['Confirmed(Indian)', 'Confirmed(Foreigner)'], var_name='Case', value_name='Count')
temp = temp.groupby(['Date', 'Case'])['Count'].sum().reset_index()
temp.head()

fig = px.bar(temp, x="Date", y="Count", color='Case', title='Cases over time',)
fig.show()

In [41]:
fig = px.bar(df, x="Date", y="Confirmed(Indian)", color='State/UT', title='Cases over time',)
fig.show()

#Soon I will be updating more on this .......